<h2 align=center> Facial Expression Recognition</h2>

- Task 1: Import Libraries
- Task 2: Plot Sample Images
- Task 3: Generate Training and Validation Batches
- Task 4: Apply CNN Model
- Task 5: Train and Evaluate Model
- Task 6: Represent Model as JSON String

## Task 1: Import Libraries

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications import resnet
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications import resnet_v2
from tensorflow.keras import regularizers
from tensorflow import keras

from IPython.display import SVG, Image
from livelossplot import PlotLossesKerasTF
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

## Task 2: Plot Sample Images

In [4]:
emotion_prop5=[]

for expression5 in os.listdir("Dataset7/train3/"):
    emotion_prop5.append(len(os.listdir("Dataset7/train3/" + expression5)))
    print(str(len(os.listdir("Dataset7/train3/" + expression5))) + " " + expression5 + " images")

print("emotion_prop4 = ",emotion_prop5)

In [5]:
emotions5 = ['happy','surprise','neutral','fear','angry','sad','disgust']
palette5 = ['gold','deepskyblue','cornflowerblue','orange','cornflowerblue','lightgreen','lightcoral']
plt.figure(figsize=[12,6])
plt.bar(x=emotions5, height=emotion_prop5, color=palette5, edgecolor='black')
plt.xlabel('Emotion')
plt.ylabel('Proportion')
plt.title('Emotion Label Proportions')
plt.show()

## Task 3: Generate Training and Validation Batches

In [6]:
# for resnet50_v2
img_size = 224
batch_size = 64

In [7]:
def to_grayscale_then_rgb(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.grayscale_to_rgb(image)
    return image

datagen_train5 = ImageDataGenerator(preprocessing_function=to_grayscale_then_rgb,
                                   rescale=1/255, rotation_range=10, brightness_range=[0.2,1.0])
train_generator5 = datagen_train5.flow_from_directory("Dataset7/train3/",
                                                    target_size=(img_size,img_size),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

datagen_validation5 = ImageDataGenerator(preprocessing_function=to_grayscale_then_rgb,
                                        rescale=1/255, rotation_range=10,brightness_range=[0.2,1.0])
validation_generator5 = datagen_validation5.flow_from_directory("Dataset7/test3/",
                                                    target_size=(img_size,img_size),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

## Task 4: Apply CNN Model

### ResNet50V2

In [11]:
# Ahmed sabry
resnet50V2_model = resnet_v2.ResNet50V2(include_top=True,
                               weights="imagenet",
                               input_shape=(img_size,img_size,3),
                               classifier_activation="softmax",
                               classes=1000)

## Apply CNN architecture

In [23]:
### # Initialising the CNN
model5 = Sequential()
model5.add(resnet50V2_model)
# Fully connected layer 1st layer
model5.add(Dense(512, kernel_regularizer=regularizers.l2(l=0.01)))
model5.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(Dropout(0.25))
# Fully connected layer 2nd layer
model5.add(Dense(256, kernel_regularizer=regularizers.l2(l=0.01)))
model5.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(Dropout(0.25))
model5.add(Dense(7, activation='softmax'))

In [24]:
import tensorflow_addons as tfa
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]
model5.compile(optimizer=Adam(lr=0.0001), loss = tfa.losses.SigmoidFocalCrossEntropy(alpha=0.20, gamma=2.0,reduction=tf.keras.losses.Reduction.AUTO), metrics=METRICS)
model5.summary()

## Task 5: Train and Evaluate Model

In [25]:
%%time

epochs5 =10
steps_per_epoch5 = train_generator5.n//train_generator5.batch_size
validation_steps5 = validation_generator5.n//validation_generator5.batch_size

#Reduce learning rate when a metric has stopped improving.
reduce_lr5 = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.000001, mode='auto')
#save the Keras model or model weights at some frequency.
checkpoint5 = ModelCheckpoint("model_weights5.h5", monitor='val_prc',
                             save_weights_only=True, mode='max', verbose=1)
#Stop training when a monitored metric has stopped improving.
early_stopp = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
#Callback
callbacks5 = [PlotLossesKerasTF(), checkpoint5, reduce_lr5,early_stopp]

In [ ]:
#Train the model
history5 = model5.fit(x = train_generator5,
                      steps_per_epoch = steps_per_epoch5,
                      epochs = epochs5,
                      validation_data = validation_generator5,
                      validation_steps = validation_steps5,
                      callbacks = callbacks5)

## Task 6: Represent Model as JSON String

In [ ]:
model_json5 = model5.to_json()
with open("./model5.json", "w") as json_file:
    json_file.write(model_json5)